In [1]:
df_ctry = spark.read.option("inferSchema","true").option("header","true").csv("/FileStore/tables/ctry_summary.csv")

In [2]:
df_ctry.createOrReplaceTempView("ctry_summary")

In [3]:
spark.sql("select * from ctry_summary").show()

In [4]:
df_sort = spark.sql("select DEST_COUNTRY_NAME,count(1) as cnt from ctry_summary group by DEST_COUNTRY_NAME sort by cnt")
#df_sort.show()
df_sort.explain()

In [5]:
df_ctry.groupBy("DEST_COUNTRY_NAME").count().explain()
#execution plan is same as SQL

In [6]:
from pyspark.sql.functions  import max 
df_ctry.select(max("count")).explain()
spark.sql("select max(count) from ctry_summary").explain()

In [7]:
df_people_flow = spark.read.option("inferSchema","true").option("header","true").csv("/FileStore/tables/global_flow_of_people.csv")

In [8]:
df_people_flow.printSchema()
df_people_flow.createOrReplaceTempView("people_flow")

In [9]:
df_sql = spark.sql("""
                   SELECT country_dest, sum(regionflow_1990) as destination_total
FROM people_flow
GROUP BY country_dest
ORDER BY sum(regionflow_1990) DESC
LIMIT 5
                   """)
df_sql.explain()

In [10]:
#withColumnRenamed
from pyspark.sql.functions import desc
df_new = df_people_flow.groupBy("country_dest")\
.sum("regionflow_1990")\
.withColumnRenamed("sum(regionflow_1990)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)

df_new.explain()

In [11]:
df_sql.collect()

In [12]:
df_new.collect()